# Installing required libraries:

In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install moviepy

Note: you may need to restart the kernel to use updated packages.


# Importing required libraries:

In [4]:
import numpy as np
import cv2
from moviepy.editor import VideoFileClip

# Lane detection script v1:

In [7]:
def process_image(image):
     gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
     blur_gray = cv2.GaussianBlur(gray, (5, 5), 0)
     edges = cv2.Canny(blur_gray, 50, 150)
    
     imshape = image.shape
     vertices = np.array([[(imshape[1] * 0.1, imshape[0]),
     (imshape[1] * 0.30, imshape[0] * 0.6),
     (imshape[1] * 0.55, imshape[0] * 0.6),
     (imshape[1] * 0.9, imshape[0])]],
     dtype=np.int32)
    
     masked_edges = region_of_interest(edges, vertices)
     lines = hough_lines(masked_edges, rho=10, theta=np.pi/180, threshold=8, min_line_len=40, max_line_gap=20)
    
     result = draw_and_fill_lane(image, lines)
     return result
def region_of_interest(img, vertices):
     mask = np.zeros_like(img)
     cv2.fillPoly(mask, vertices, 255)
     masked_img = cv2.bitwise_and(img, mask)
     return masked_img
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
     lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
     return lines
def draw_and_fill_lane(img, lines, color=[255, 0, 0], thickness=8, fill_color=[0, 255, 0]):
     line_img = np.zeros_like(img)
     draw_lines(line_img, lines, color, thickness)
    
     left_lines, right_lines = separate_lines(lines)
     fill_lane_area(line_img, left_lines, right_lines, fill_color)
    
     result = weighted_img(line_img, img)
     return result

def draw_lines(img, lines, color, thickness):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
def separate_lines(lines):
     left_lines = []
     right_lines = []
    
     for line in lines:
         for x1, y1, x2, y2 in line:
             slope = (y2 - y1) / (x2 - x1)
             if slope < -0.5: # Left lane line
                 left_lines.append(line)
             elif slope > 0.5: # Right lane line
                 right_lines.append(line)
        
     return left_lines, right_lines
def fill_lane_area(img, left_lines, right_lines, fill_color):
     if left_lines and right_lines: # Check if both left and right lines are present
         pts_left = np.array([[x1, y1] for line in left_lines for x1, y1, x2, y2 in line] +
                             [[x2, y2] for line in left_lines for x1, y1, x2, y2 in line])
         pts_right = np.array([[x1, y1] for line in right_lines for x1, y1, x2, y2 in line] +
                              [[x2, y2] for line in right_lines for x1, y1, x2, y2 in line])
         pts_right = pts_right[::-1]
         pts = np.vstack((pts_left, pts_right))
         cv2.fillPoly(img, [pts], fill_color)
def weighted_img(img, initial_img, α=0.8, β=1.0, λ=0.0):
     return cv2.addWeighted(initial_img, α, img, β, λ)
# Process video
input_path = "C:\\Users\\DSK 8920444598\\Videos\\lane_detection\\input1.mp4"
output_path = "C:\\Users\\DSK 8920444598\\Videos\\lane_detection\\output_1b.mp4"
clip = VideoFileClip(input_path)
processed_clip = clip.fl_image(process_image)
processed_clip.write_videofile(output_path, audio=False)

Moviepy - Building video C:\Users\DSK 8920444598\Videos\lane_detection\output_1b.mp4.
Moviepy - Writing video C:\Users\DSK 8920444598\Videos\lane_detection\output_1b.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\DSK 8920444598\Videos\lane_detection\output_1b.mp4


# Lane detection script v2:

In [8]:
import cv2
import numpy as np
from moviepy.editor import VideoFileClip

def process_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur_gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur_gray, 50, 150)
    
    imshape = image.shape
    vertices = np.array([[(imshape[1] * 0.1, imshape[0]),
                          (imshape[1] * 0.45, imshape[0] * 0.6),
                          (imshape[1] * 0.55, imshape[0] * 0.6),
                          (imshape[1] * 0.9, imshape[0])]],
                        dtype=np.int32)
    
    masked_edges = region_of_interest(edges, vertices)
    lines = hough_lines(masked_edges, rho=1, theta=np.pi/180, threshold=15, min_line_len=20, max_line_gap=10)
    
    result = draw_and_fill_lane(image, lines)
    return result

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, 255)
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

def draw_and_fill_lane(img, lines, color=[255, 0, 0], thickness=8, fill_color=[0, 255, 0]):
    line_img = np.zeros_like(img)
    draw_lines(line_img, lines, color, thickness)
    
    left_lines, right_lines = separate_lines(lines)
    fill_lane_area(line_img, left_lines, right_lines, fill_color)
    
    result = weighted_img(line_img, img)
    return result

def draw_lines(img, lines, color, thickness):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def separate_lines(lines):
    left_lines = []
    right_lines = []
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1)
            if slope < -0.5 and slope > -2: # Adjusted slope range for left lane line
                left_lines.append(line)
            elif slope > 0.5 and slope < 2: # Adjusted slope range for right lane line
                right_lines.append(line)
                
    return left_lines, right_lines

def fill_lane_area(img, left_lines, right_lines, fill_color):
    if left_lines and right_lines: # Check if both left and right lines are present
        pts_left = np.array([[x1, y1] for line in left_lines for x1, y1, x2, y2 in line] +
                             [[x2, y2] for line in left_lines for x1, y1, x2, y2 in line])
        pts_right = np.array([[x1, y1] for line in right_lines for x1, y1, x2, y2 in line] +
                              [[x2, y2] for line in right_lines for x1, y1, x2, y2 in line])
        pts_right = pts_right[::-1]
        pts = np.vstack((pts_left, pts_right))
        cv2.fillPoly(img, [pts], fill_color)

def weighted_img(img, initial_img, α=0.8, β=1.0, λ=0.0):
    return cv2.addWeighted(initial_img, α, img, β, λ)

# Process video
input_path = "C:\\Users\\DSK 8920444598\\Videos\\lane_detection\\test5.mp4"
output_path = "C:\\Users\\DSK 8920444598\\Videos\\lane_detection\\test_5b.mp4"
clip = VideoFileClip(input_path)
processed_clip = clip.fl_image(process_image)
processed_clip.write_videofile(output_path, audio=False)


Moviepy - Building video C:\Users\DSK 8920444598\Videos\lane_detection\test_5b.mp4.
Moviepy - Writing video C:\Users\DSK 8920444598\Videos\lane_detection\test_5b.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\DSK 8920444598\Videos\lane_detection\test_5b.mp4
